<a href="https://colab.research.google.com/github/Real1CM/ComputerVison/blob/main/2024_10_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 图像的基本运算-加减法
###### 8号 梁宸铭 202408120081142

######1. 从视频文件中截取两张位置相邻的图片。提示：使用cv2.imread()函数读取一段视频文件。将两幅图片做差后取绝对值，显示第一张、第二张图片做差后的图像，讨论能否用此算法进行视频监控，发现场景中的异常情况，会遇到哪些问题？

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

video_path = './video.mp4'
cap = cv2.VideoCapture(video_path)

ret1, frame1 = cap.read()
ret2, frame2 = cap.read()

diff = cv2.absdiff(frame1, frame2)

cv2_imshow(frame1)
cv2_imshow(frame2)
cv2_imshow(diff)

当出现物体运动比如：光线变化，树叶摇晃等可能出现误报。除非背景是全部不动的此方法可行。

######2. 将两张彩色图片A，B使用加法运算进行融合，融合后的图像C=aA+(1-a)B其中0<=a<=1，改变参数a，分析融合后图片C的变化。

In [ ]:
imageA = cv2.imread('./cat.jpg')
imageB = cv2.imread('./dog.jpg')

if imageA.shape != imageB.shape:
    imageB = cv2.resize(imageB, (imageA.shape[1], imageA.shape[0]))

a = 0.7
C = cv2.addWeighted(imageA, a, imageB, 1 - a, 0)
cv2_imshow(C)

In [ ]:
a = 0.3
C = cv2.addWeighted(imageA, a, imageB, 1 - a, 0)
cv2_imshow(C)

a值越靠近1，图像越像A，a值越靠近0，图像越像B

######3. 选择的一张图片使用Gamma函数对图像进行算法实验，验证通过设置不同的参数y=0.5，1.5，2.5，分别增强了图像中的哪些信息，并讨论增强后的效果。

In [ ]:
image = cv2.imread('./cat.jpg')

def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

gamma_values = [0.5, 1.5, 2.5]

for gamma in gamma_values:
    adjusted = adjust_gamma(image, gamma)
    cv2_imshow(adjusted)

数字越小越增强亮的地方，比如猫脸。越大增强暗的地方，比如背景。

######4. 尝试寻找某种数据(图像)，尝试使用log变换来对图像中的信息进行增强，对比增强前后的效果，说明使用log变换后图片的质量有明显提高。

In [ ]:
image = cv2.imread('./cat.jpg')

normalized_image = image / 255.0
c = 255 / np.log(1 + np.max(normalized_image))
log_transformed_image = c * np.log(1 + normalized_image)
log_transformed_image = np.uint8(log_transformed_image)
cv2_imshow(log_transformed_image)

log函数值越小变化越大，gamma矫正是x^n函数，变化和n有关，gamma矫正更加灵活通过调整n来特别地增强暗部或亮部。

######5. 对上面的图片直接使用直方图均衡算法，并画出各个算法处理后图像的直方图，讨论哪个算法处理后的效果在视觉上比较好，可以从视觉直观效果和直方图等方面进行综合评价。

In [ ]:
from matplotlib import pyplot as plt

image = cv2.imread('./cat.jpg')

def apply_log_transformation(image):
    c = 255 / np.log(1 + np.max(image))
    log_image = c * (np.log(1 + image))
    log_image = np.array(log_image, dtype=np.uint8)
    return log_image

def apply_gamma_correction(image, gamma):
    inv_gamma = 1.0 / gamma
    gamma_image = np.array(255 * (image / 255) ** inv_gamma, dtype='uint8')
    return gamma_image

def apply_hist_equalization(image):
    b, g, r = cv2.split(image)
    b_eq = cv2.equalizeHist(b)
    g_eq = cv2.equalizeHist(g)
    r_eq = cv2.equalizeHist(r)
    equalized_image = cv2.merge([b_eq, g_eq, r_eq])
    return equalized_image

def plot_histogram(image, title):
    color = ('b', 'g', 'r')
    plt.figure()
    for i, col in enumerate(color):
        hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        plt.plot(hist, color=col)
        plt.xlim([0, 256])
    plt.title(title)
    plt.show()

plot_histogram(image, "1")
log_image = apply_log_transformation(image)
plot_histogram(log_image, "2")
gamma_image = apply_gamma_correction(image, 0.5)
plot_histogram(gamma_image, "3")
equalized_image = apply_hist_equalization(image)
plot_histogram(equalized_image, "4")

cv2_imshow(image)
cv2_imshow(log_image)
cv2_imshow(gamma_image)
cv2_imshow(equalized_image)

Gamme矫正更加灵活，就这个图片而言Gamme矫正更好，直方图均衡让图像变得模糊了。